# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3
import nltk
import re
import os
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

nltk.download(['punkt', 'wordnet']);


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Q524307\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Q524307\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///ETL_Data.db')
df = pd.read_sql('ETL_Data', engine)


# seperate dataset to input variables (X) and target variable (Y)
X = df.message
Y = df[df.columns[4:]]

df


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    Takes a text string as input and performs several text preprocessing steps on it including:
    - replacing URLs in the text with a placeholder
    - tokenizing the text into individual words
    - lemmatizing the words to their base form
    - converting the words to lowercase
 
    Arguments:
    text -- the input text string to be processed

    Returns:
    clean_tokens -- a list of cleaned tokens extracted from the input text string
    """

    # replace all urls wirth a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # extract urls from given text
    detected_urls = re.findall(url_regex, text)

    # replace all urls to 'url'
    for url in detected_urls:
        text = text.replace(url, "url")

    # extract the word tokens
    tokens = word_tokenize(text)

    # lemmatize to the stem of a word
    lemmatizer = WordNetLemmatizer()
    
    # list clean tokens
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# define the pipeline
# this pipeline can be trained on a dataset of labeled messages and used to predict the category of new, unlabeled messages based on the trained model.
pipeline = Pipeline([
        # counts the number of occurrences of each word in a text message
        # it takes an optional tokenizer parameter, which in this case is a custom tokenize function that splits a text message into individual words.
        ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)),
        # computes the TF-IDF (term frequency-inverse document frequency) scores for the words in the text messages
        # this is a way of weighing the importance of each word based on how frequently it appears in the messages 
        # and how common it is across all messages in the dataset.
        ('tfidf', TfidfTransformer()),
        # type of machine learning classifier that uses a collection of decision trees to classify messages into different categories
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)


KeyboardInterrupt: 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)


def class_report(y_test, y_pred):
   """
   Prints the classification report containing precision, recall, f1 score and support 
   for each label in y_test and y_pred.
   
   Args:
   y_test (pandas.DataFrame): DataFrame with true labels.
   y_pred (numpy.ndarray): NumPy array with predictions generated by a trained model.
   
   Returns:
   None
   """
   # Loop through each column of y_test
   for i, col in enumerate(y_test.columns):
       print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
       # Print the name of the column
       print(col, ':')
       print('-----------------------------------------------------')
       # Print the classification report for y_test and y_pred at ith column
       print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

# Call the class_report function with y_test and y_pred to print the classification report
class_report(y_test, y_pred)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# display pipelines parameters
pipeline.get_params()

In [ ]:
parameters = {
    'tfidf__norm':['l2','l1'],
    'clf__estimator__min_samples_split':[2,3],
}

cv = GridSearchCV(pipeline, parameters)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.